# Análisis del riesgo de incumplimiento de los prestatarios

## Propósito del proyecto:
    
<div style="text-align: justify">
El propósito de este proyecto es realizar un análisis del riesgo de incumplimiento de los prestatarios en la división de préstamos de un banco. El objetivo principal es determinar si el estado civil y el número de hijos de un cliente tienen un impacto significativo en el incumplimiento de pago de un préstamo. El informe resultante será utilizado para desarrollar una puntuación de crédito que permita evaluar la capacidad de pago de los prestatarios potenciales.
</div>

## Hipótesis a evaluar:

<div style="text-align: justify">
En el marco de este proyecto, se evaluarán las siguientes hipótesis:
</div>

**Hipótesis** **1:** 
El estado civil de un cliente tiene un impacto en el incumplimiento de pago de un préstamo. Se sospecha que los clientes solteros o divorciados pueden tener una mayor probabilidad de incumplimiento en comparación con los clientes casados o en una relación estable.

**Hipótesis** **2:**
El número de hijos de un cliente influye en el incumplimiento de pago de un préstamo. Se plantea la posibilidad de que los clientes con un mayor número de hijos tengan una mayor probabilidad de incumplimiento debido a la carga financiera adicional asociada con la crianza de los hijos.

Estas hipótesis serán examinadas y analizadas utilizando técnicas de análisis de datos con el fin de obtener conclusiones y recomendaciones para la creación de una puntuación de crédito efectiva.
</div>

## Abre el archivo de datos y mira la información general. 

In [1]:
# Cargar todas las librerías

import pandas as pd

# Carga los datos

data_report = pd.read_csv("/datasets/credit_scoring_eng.csv")

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [2]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos

data_report.shape

(21525, 12)

In [3]:
# vamos a mostrar las primeras filas N

data_report.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Al analizar la muestra de datos, se identificaron aspectos que requieren atención y cambios en el proceso de análisis:
       
> La columna **"days_employed"** presenta valores negativos y ausentes. Se recomienda investigar su origen y determinar su relevancia para el análisis. Si no es relevante, se sugiere eliminarla. En caso de ser importante, es necesario abordar adecuadamente los valores negativos y ausentes.
    
> En la columna **"education"** se encontraron errores de escritura. Se recomienda corregirlos utilizando una función de limpieza de datos, como el método replace() de Pandas, para estandarizar los valores.
  
> La columna **"total_income"** tiene valores ausentes. Es fundamental tratar estos valores faltantes, considerando enfoques como el relleno con la media o mediana de la columna.
 
> En la columna **"purpose"** se detectaron datos duplicados. Se sugiere aplicar una función de limpieza de datos para corregirlos y garantizar categorías únicas en el análisis.

Estos hallazgos resaltan la importancia de realizar una limpieza y preparación adecuada de los datos antes del análisis. Al abordar estos problemas, se mejorará la calidad y confiabilidad de los resultados obtenidos.

In [4]:
# Obtener información sobre los datos

data_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<div style="text-align: justify">
Se observa que hay valores ausentes en dos columnas: "days_employed" y "total_income". Esto implica que estas dos columnas tienen registros faltantes en comparación con el número total de filas en el DataFrame. En cambio, el resto de las columnas parecen tener datos completos.
</div>

In [5]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

missing_data_report = data_report[data_report["days_employed"].isnull()]
missing_data_report

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


<div style="text-align: justify">
La presencia de valores ausentes en ambas columnas, "days_employed" y "total_income", sugiere una posible relación o patrón entre estas dos variables. Es necesario investigar más a fondo esta relación y examinar si hay alguna explicación lógica para esta simetría. Por ejemplo, la falta de datos en la columna "days_employed", que representa la experiencia laboral, podría estar relacionada con la falta de datos en la columna "total_income", que refleja los ingresos mensuales. Es importante realizar un análisis más detallado de los datos y considerar otras variables relevantes para comprender mejor esta coincidencia y su impacto en el análisis del riesgo de incumplimiento de pago de préstamos.
</div>

In [6]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

filtered_data_report = data_report[(data_report['days_employed'].isna()) & (data_report['total_income'].isna())]
len(filtered_data_report)

2174

**Conclusión intermedia**


El número de filas en la tabla filtrada **(2174)** coincide con el número de valores ausentes en las columnas "days_employed" y "total_income", esta coincidencia nos indica la importancia de investigar más a fondo la relación entre las variables. Es posible que haya una razón común detrás de los valores ausentes, como los clientes que no proporcionaron información sobre su empleo y, como resultado, no se dispone de datos sobre sus ingresos mensuales.

En relación a las conclusiones obtenidas hasta ahora, los próximos pasos consisten en analizar la relación entre una característica definida de cliente, según la cantidad de valores nulos de la columna primera columna con valores faltantes, además, es necesario verificar si los valores ausentes dependen de otros indicadores en las demás columnas específicas de características de los clientes.

Dependiendo de los resultados, podríamos considerar técnicas de imputación para los valores faltantes o tomar otras medidas para abordar los valores nulos de manera precisa.

In [7]:
# Calcula el porcentaje de los valores ausentes en comparación con el conjunto de datos completo.   

total_data = data_report.shape [0]
total_missings = len(filtered_data_report)

percentage_missing = (total_missings / total_data) * 100
("Porcentaje de valores ausentes: {:.2f}%".format(percentage_missing))

'Porcentaje de valores ausentes: 10.10%'

<div style="text-align: justify">
Aunque el porcentaje de valores ausentes en las columnas "total_income" y "days_employed", no es considerablemente grande (10,10%), en el contexto de este proyecto de análisis de crédito, se requiere precaución al procesar estos datos y se recomienda verificar otras variables para explorar posibles relaciones antes de tomar decisiones sobre cómo manejar los valores ausentes.  Para esto vamos a empezar analizando la relacion entre la característica "income_type" para determinar si está relacionada con la falta de datos.    
</div>

In [8]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

filtered_income_type = missing_data_report['income_type']
filtered_income_type

12             retiree
26       civil servant
29             retiree
41       civil servant
55             retiree
             ...      
21489         business
21495         employee
21497         business
21502         employee
21510         employee
Name: income_type, Length: 2174, dtype: object

In [9]:
# Comprobación de la distribución

categories = ["education", "family_status", "gender", "income_type", "purpose" ]

for category in categories:
    print (missing_data_report[category]. value_counts ())

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64
married              1237
civil partnership     442
unmarried             288
divorced              112
widow / widower        95
Name: family_status, dtype: int64
F    1484
M     690
Name: gender, dtype: int64
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64
having a wedding                            92
to have a wedding                           81
wedding ceremony                            76
construction of own property                75
housing transactions                        74
buy real estate                             72

**Posibles razones por las que hay valores ausentes en los datos**
    
<div style="text-align: justify">
Después de analizar la distribución de valores ausentes en la columna "income_type", se observó que alrededor del 60% de los datos faltantes corresponden a las categorías "employee" y "business". Esto indica que la estabilidad laboral de los clientes puede estar relacionada con los datos ausentes en las columnas "days_employed" y "total_income". Sin embargo, se requiere un análisis más detallado para comprender completamente las razones detrás de estos valores ausentes y considerar otras características relevantes. 
</div>

In [10]:
# Comprobando la distribución en el conjunto de datos entero

for category in categories:
    print(data_report[category].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
GRADUATE DEGREE            1
Graduate Degree            1
Name: education, dtype: int64
married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64
F      14236
M       7288
XNA        1
Name: gender, dtype: int64
employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name

**Conclusión intermedia**

<div style="text-align: justify">
Al analizar la distribución de valores entre el conjunto de datos original y la tabla filtrada con valores ausentes, se observa una tendencia donde las categorías con más registros en determinadas variables en el conjunto de datos original muestran una mayor proporción de valores ausentes en la tabla filtrada. En particular, en la categoría "educación", se nota que las variables con más registros, como "secondary education" y "bachelor's degree", presentan una proporción más alta de valores ausentes en la tabla filtrada. El estudio de las variables de educación, estado civil y género nos brindará información relevante para determinar si existen patrones significativos en los valores ausentes.
</div>

In [11]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes

Test_1 = pd.pivot_table(missing_data_report, values='income_type', aggfunc='count', index=["education"], columns=['family_status'], fill_value = 0, margins = True)
Test_1

family_status,civil partnership,divorced,married,unmarried,widow / widower,All
education,,,,,,
BACHELOR'S DEGREE,5,1,14,3,0,23
Bachelor's Degree,8,2,11,4,0,25
PRIMARY EDUCATION,0,0,1,0,0,1
Primary Education,0,0,0,0,1,1
SECONDARY EDUCATION,18,3,38,4,4,67
SOME COLLEGE,2,1,1,3,0,7
Secondary Education,16,4,35,5,5,65
Some College,2,0,3,2,0,7
bachelor's degree,89,21,281,93,12,496


**Conclusión intermedia**

<div style="text-align: justify">
Los valores ausentes en nuestro conjunto de datos presentan un patrón discernible. Aproximadamente el 77% de los valores ausentes corresponden principalmente a clientes con educación secundaria seguidas en menor proporcion por aquellos con bachelor's degree, que están casados o en una unión libre. Esta observación indica que ciertas características demográficas están asociadas con la presencia de valores ausentes. Es importante considerar este patrón al realizar el análisis del riesgo de incumplimiento de los prestatarios, ya que podría haber un sesgo en los datos que afecte la precisión de nuestras conclusiones.
</div>

In [12]:
# Comprobación de otros patrones: explica cuáles

gender_missing_data = missing_data_report.groupby("gender")["income_type"].value_counts()
income_type_report = filtered_income_type.value_counts()
female_income_type_report = gender_missing_data.loc["F"]
male_income_type_report = gender_missing_data.loc["M"]

comparative_missing_data = pd.concat([income_type_report, female_income_type_report, male_income_type_report], axis=1, keys=['Total', 'Female','Male'])
comparative_missing_data.fillna(0, inplace=True)

print("Reporte comparativo del recuento de valores ausentes por tipo de ingreso:")
comparative_missing_data

Reporte comparativo del recuento de valores ausentes por tipo de ingreso:


,Total,Female,Male
employee,1105,693.0,412
business,508,327.0,181
retiree,413,352.0,61
civil servant,147,112.0,35
entrepreneur,1,0.0,1


**Conclusiones**

<div style="text-align: justify">
Después de analizar los datos de valores ausentes en las variables de educación, estado civil y género, se encontraron patrones significativos. Se observó que la categoría "secondary education" y el estado civil, estaban fuertemente relacionados con la presencia de valores ausentes. Además, al segmentar los datos por género, se encontró que la mayoría de los valores ausentes correspondían a mujeres. Estos hallazgos sugieren que los valores ausentes no son aleatorios, sino que siguen un patrón consistente. Específicamente, las mujeres con estado civil comprometidas y educación secundaria tienen una mayor propensión a tener valores ausentes. 
</div>


  
> En resumen, se identificaron patrones significativos en los valores ausentes relacionados con la educación, estado civil y género. Estos patrones proporcionan información valiosa para comprender la naturaleza de los datos faltantes y deben considerarse en el análisis posterior de los datos. Es necesario profundizar en la investigación para comprender las causas subyacentes de estos patrones y evaluar posibles sesgos en la recopilación de datos o diferencias en la disponibilidad y precisión de la información según la categoría de educación y estado civil.
</div>

<div style="text-align: justify">
Para abordar los valores ausentes, se aplicará un enfoque de imputación de datos utilizando la mediana de los ingresos de la columna "total_income" según el género. Dado que los valores ausentes se encuentran en columnas relacionadas con los ingresos y son de importancia para realizar un análisis crediticio, es crucial tratarlos adecuadamente. Al utilizar la mediana, se busca evitar posibles sesgos causados por valores atípicos y proporcionar una estimación razonable de los ingresos faltantes. 
</div>


      
> Al calcular la mediana por separado para hombres y mujeres, se tiene en cuenta la posible disparidad de ingresos entre géneros y se busca capturar mejor la distribución de los ingresos correspondiente a cada grupo. Por otra parte los datos ausentes en la columna "days_employed" seran analizados posteriormente.
</div>

<div style="text-align: justify">
En los próximos pasos de la transformación de datos, eliminaré duplicados para garantizar la integridad de los datos. Además, corregiré registros inconsistentes y artefactos incorrectos para mejorar la precisión de la información. Asimismo, utilizaré agrupaciones por grupos de edades para obtener una perspectiva más amplia y comprender mejor los patrones en los datos.
</div>

## Transformación de datos


<div style="text-align: justify">
Vamos a usar el método unique() en la columna "education" para obtener los valores únicos y revisar si hay datos duplicados o que requieran corrección debido a posibles errores humanos. Esto nos permitirá identificar cualquier problema en los valores y tomar las medidas necesarias para corregirlos.
</div>

In [13]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente

data_report['education'].unique().tolist()

["bachelor's degree",
 'secondary education',
 'Secondary Education',
 'SECONDARY EDUCATION',
 "BACHELOR'S DEGREE",
 'some college',
 'primary education',
 "Bachelor's Degree",
 'SOME COLLEGE',
 'Some College',
 'PRIMARY EDUCATION',
 'Primary Education',
 'Graduate Degree',
 'GRADUATE DEGREE',
 'graduate degree']

In [14]:
# Arregla los registros si es necesario

primary_duplicates = ['primary education','PRIMARY EDUCATION']
primary_name ='Primary Education'
secondary_duplicates = ['secondary education','SECONDARY EDUCATION']
secondary_name = 'Secondary Education'
bachelor_duplicates = ["bachelor's degree","BACHELOR'S DEGREE", "Bachelor's Degree"]
bachelor_name = "Bachelor Degree"
some_college_duplicates= ['some college', 'SOME COLLEGE']
some_college_name = 'Some College'
graduate_duplicates = ['graduate degree', 'GRADUATE DEGREE']
graduate_name = 'Graduate Degree'

def replace_wrong_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        data_report['education'] = data_report['education'].replace(wrong_value, correct_value)

replace_wrong_values(primary_duplicates, primary_name)
replace_wrong_values(secondary_duplicates, secondary_name)
replace_wrong_values(bachelor_duplicates, bachelor_name)
replace_wrong_values(some_college_duplicates, some_college_name)
replace_wrong_values(graduate_duplicates, graduate_name)  
    
data_report ['education']

0            Bachelor Degree
1        Secondary Education
2        Secondary Education
3        Secondary Education
4        Secondary Education
                ...         
21520    Secondary Education
21521    Secondary Education
21522    Secondary Education
21523    Secondary Education
21524    Secondary Education
Name: education, Length: 21525, dtype: object

In [15]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido

data_report['education'].unique().tolist()

['Bachelor Degree',
 'Secondary Education',
 'Some College',
 'Primary Education',
 'Graduate Degree']

Continuaremos verificando nuestros datos, ahora centrándonos en la columna "children". Revisaremos estos valores para identificar cualquier dato extraño, como valores negativos o atípicos que no se ajusten a la naturaleza de la variable

In [16]:
# Veamos la distribución de los valores en la columna `children`

data_report ["children"].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

<div style="text-align: justify">
En la columna "children" encontramos valores problemáticos, como -1 y 20, que no parecen ser válidos para indicar la cantidad de hijos. Estos valores representan aproximadamente el 25% de los datos en esta columna y podrían ser el resultado de errores de tipeo o ingreso incorrecto de datos. Para abordar esta situación, utilizaré el método .loc para seleccionar los registros específicos que contienen estos valores problemáticos y les asignaré nuevos valores.
</div>

> Para los valores negativos, los cambiaré a 0 asumiendo que fueron errores de tipeo, ya que no es posible tener una cantidad negativa de hijos. Para el valor 20, consideraré que fue ingresado incorrectamente y lo cambiaré a un valor más realista, como 2 o 3, basándome en la distribución de valores cercanos.

In [17]:
# [arregla los datos según tu decisión]

data_report.loc[data_report['children'] == -1, 'children'] = 0
data_report.loc[data_report['children'] == 20, 'children'] = 2
data_report['children'] = data_report['children'].astype('int16')
data_report['children']

0        1
1        1
2        0
3        3
4        0
        ..
21520    1
21521    0
21522    1
21523    3
21524    2
Name: children, Length: 21525, dtype: int16

In [18]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

data_report["children"].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

<div style="text-align: justify">
Dado que la columna days_employed contiene datos numéricos, no utilizaremos el método unique() debido a que los datos únicos podrían ser demasiado extensos para revisar. En su lugar, realizaremos una revisión de la columna con el método describe() para obtener una idea de la naturaleza y distribución de los datos. Esta revisión nos permitirá identificar posibles problemas o patrones iniciales en los datos de days_employed.
</div>

In [19]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje

print (data_report['days_employed'].describe())

days_employed_problematic_data = len (data_report['days_employed'].value_counts())
percentage_days_employed_problematic_data = (days_employed_problematic_data / total_data) * 100
("Porcentaje de datos problematicos en `days_employed`: {:.2f}%".format(percentage_days_employed_problematic_data))

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


'Porcentaje de datos problematicos en `days_employed`: 89.90%'

<div style="text-align: justify">
Los datos problemáticos en la columna "days_employed" representan un porcentaje significativo del total de datos, lo que sugiere posibles problemas técnicos. Al revisar los estadísticos descriptivos, se observa que los valores positivos presentan cantidades irrealmente altas en el contexto de "días laborados". Además, también se identifican valores negativos.
</div>


   Para abordar estos problemas, se propone seguir el siguiente flujo de trabajo:
- Identificar los valores positivos y negativos en la columna "days_employed" mediante filtrado.
- Resolver el problema de magnitud de los valores positivos, ya sea dividiendo por una constante, reemplazándolos por NaN u aplicando otra corrección adecuada.
- Convertir los valores negativos a positivos utilizando el valor absoluto.

In [20]:
# Aborda los valores problemáticos, si existen.

import numpy as np
data_report.loc[data_report['days_employed'] > 0, 'days_employed'] = np.nan

data_report.loc[data_report['days_employed'] < 0, 'days_employed'] = data_report.loc[data_report['days_employed'] < 0, 'days_employed'].abs()
data_report['days_employed']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4                NaN
            ...     
21520    4529.316663
21521            NaN
21522    2113.346888
21523    3112.481705
21524    1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [21]:
# Comprueba el resultado - asegúrate de que esté arreglado

data_report['days_employed'].describe()

count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64

<div style="text-align: justify">
En la columna "dob_years", que representa la edad de los clientes, analizaremos los datos estadísticos descriptivos. Este análisis nos permitirá evaluar si existen datos inusuales o inconsistentes en la columna de edad de los clientes. De ser así, calcularemos el porcentaje correspondiente de valores incoherentes.
    </div>

In [22]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje

data_report['dob_years'].value_counts().sort_index()

dob_years_suspect_data = len(data_report[data_report['dob_years'] < 18])
percentage_dob_years_suspect_data = (dob_years_suspect_data / total_data) * 100
("Porcentaje de datos sospechosos en 'dob_years': {:.2f}%".format(percentage_dob_years_suspect_data))

"Porcentaje de datos sospechosos en 'dob_years': 0.47%"

<div style="text-align: justify">
En este caso, los valores sospechosos corresponden a registros donde la edad registrada es menor a 18 años. Al corregir estos valores problemáticos, se mejora la calidad de los datos, esta decisión radica en el hecho de que es poco probable que los clientes sean menores de edad en un contexto crediticio. Al marcar los valores sospechosos como NaN en lugar de 0, se podrá realizar un proceso de imputación de valores ausentes, procediendo a rellenar los valores NaN por la media de edad.
</div>

In [23]:
# Resuelve los problemas en la columna `dob_years`, si existen

data_report.loc[data_report['dob_years'] < 18, 'dob_years'] = np.nan
mean_dob_years = data_report['dob_years'].mean()
data_report['dob_years'] = data_report['dob_years'].fillna(mean_dob_years).astype("int16")
data_report['dob_years']

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: dob_years, Length: 21525, dtype: int16

In [24]:
# Comprueba el resultado - asegúrate de que esté arreglado

data_report['dob_years'].value_counts().sort_index().head()

19     14
20     51
21    111
22    183
23    254
Name: dob_years, dtype: int64

En la columna "family_status", podemos revisar los diferentes tipos de valores presentes y determinar si hay algún problema que debamos abordar

In [25]:
# Veamos los valores de la columna

data_report["family_status"].unique().tolist()

['married', 'civil partnership', 'widow / widower', 'divorced', 'unmarried']

In [26]:
# Aborda los valores problemáticos en `family_status`, si existen

data_report["family_status"] = data_report["family_status"].str.title()

In [27]:
# Comprueba el resultado - asegúrate de que esté arreglado

data_report["family_status"].unique().tolist()

['Married', 'Civil Partnership', 'Widow / Widower', 'Divorced', 'Unmarried']

Veamos los valores únicos en la columna "gender" para identificar posibles problemas:

In [28]:
# Veamos los valores en la columna

data_report["gender"].unique().tolist()

['F', 'M', 'XNA']

In [29]:
# Aborda los valores problemáticos, si existen

print (data_report["gender"].value_counts())
data_report = data_report[data_report["gender"] != "XNA"]

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [30]:
# Comprueba el resultado - asegúrate de que esté arreglado

data_report["gender"].unique().tolist()

['F', 'M']

Realizaremos un análisis de la columna "income_type" para examinar e identificar posibles problemas.

In [31]:
# Veamos los valores en la columna

data_report ["income_type"].unique().tolist()

['employee',
 'retiree',
 'business',
 'civil servant',
 'unemployed',
 'entrepreneur',
 'student',
 'paternity / maternity leave']

In [32]:
# Aborda los valores problemáticos, si existen

data_report ["income_type"] = data_report ["income_type"].str.title()

In [33]:
# Comprueba el resultado - asegúrate de que esté arreglado

data_report ["income_type"].unique().tolist()

['Employee',
 'Retiree',
 'Business',
 'Civil Servant',
 'Unemployed',
 'Entrepreneur',
 'Student',
 'Paternity / Maternity Leave']

<div style="text-align: justify">
Es necesario verificar si existen duplicados en los datos y decidir cómo manejarlos. Esto implica determinar si se deben eliminar, mantener solo una instancia de cada duplicado o realizar una investigación adicional antes de tomar una decisión. El enfoque elegido dependerá del contexto y los objetivos del análisis.
    </div>

In [34]:
# Comprobar los duplicados

data_report.duplicated().sum()

71

In [35]:
# Aborda los duplicados, si existen

data_report = data_report.drop_duplicates().reset_index(drop=True)

In [36]:
# Última comprobación para ver si tenemos duplicados

data_report.duplicated().sum()

0

In [37]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones

data_report.shape

(21453, 12)

<div style="text-align: justify">
Después de realizar las manipulaciones iniciales en el conjunto de datos, se han realizado cambios en varias columnas, como la eliminación de valores negativos en "days_employed", la corrección de valores sospechosos en "dob_years", la capitalización de las palabras en "family_status" y "income_type", y la eliminación de filas duplicadas. En términos porcentuales, los cambios realizados representan aproximadamente el 0.33% del conjunto de datos total. Estos cambios han sido relativamente mínimos en relación con el tamaño y la integridad general del conjunto de datos original.
</div>

# Trabajar con valores ausentes

<div style="text-align: justify">
En ocasiones, es útil trabajar con diccionarios cuando se manejan valores identificados por IDs, ya que permiten establecer una relación entre los IDs y su información correspondiente, traduciendo estos datos en valores más comprensibles y agilizando las operaciones relacionadas. Trabajar con diccionarios mejora la eficiencia y la legibilidad al acceder y manipular datos que utilizan IDs para representar cierta información.
</div> 
      
> En nuestro conjunto de datos, hemos identificado dos columnas que contienen IDs: "education_id" y "family_status_id". Estos IDs están asociados a los valores de las columnas "education" y "family_status", respectivamente. Para facilitar el análisis y la interpretación de estos datos, utilizaremos diccionarios para mapear los IDs a los valores correspondientes.

In [38]:
# Encuentra los diccionarios

# Diccionario para la columna "education_id"
education_dict = data_report[['education_id', 'education']].drop_duplicates().set_index('education_id')['education'].to_dict()

# Diccionario para la columna "family_status_id"
family_status_dict = data_report[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')['family_status'].to_dict()

### Restaurar valores ausentes en `total_income`

<div style="text-align: justify">
En nuestro conjunto de datos, hemos identificado dos columnas con valores ausentes: "days_employed" y "total_income". Para abordar estos valores faltantes, utilizaremos una estrategia de imputación que nos permita preservar la distribución y la tendencia central de los datos.
 </div> 
<div style="text-align: justify">  
Para abordar los valores ausentes de "total_income", aplicaré una estrategia basada en la categorización de edad de los clientes. Para ello, creamos una nueva columna llamada "age_category" utilizando la información de la columna "dob_years". Esta categorización nos permitirá calcular valores estimados para el ingreso total utilizando los datos disponibles dentro de cada grupo de edad.
     </div> 

In [39]:
# Vamos a escribir una función que calcule la categoría de edad

def age_group (age):
    
    # "Young" for age < 30
    # "Adult" for age < 60
    # "Senior" for all other cases
    
    if age < 30:
        return "Young"
    
    elif age < 60:
        return "Adult"
    
    return "Senior"

In [40]:
# Prueba si la función funciona bien

print(age_group (25))
print(age_group (40))
print(age_group (70))

Young
Adult
Senior


In [41]:
# Crear una nueva columna basada en la función

data_report ["age_category"] = data_report["dob_years"].apply(age_group)
data_report ["age_category"]

0         Adult
1         Adult
2         Adult
3         Adult
4         Adult
          ...  
21448     Adult
21449    Senior
21450     Adult
21451     Adult
21452     Adult
Name: age_category, Length: 21453, dtype: object

In [42]:
# Comprobar cómo es la distribución de los valores en la nueva columna

data_report['age_category'].value_counts()

Adult     15774
Young      3179
Senior     2500
Name: age_category, dtype: int64

<div style="text-align: justify">
En general, los factores que suelen influir en los ingresos de una persona incluyen la educación, el tipo de empleo y el género, ya que existen disparidades salariales entre hombres y mujeres en muchos contextos laborales. Para determinar si debemos usar valores medios o medianos para reemplazar los valores ausentes en la columna de ingresos, es importante analizar la distribución de estos factores y su relación con los ingresos.
 </div>

<div style="text-align: justify">
Crearemos un nuevo DataFrame que contendrá únicamente las filas que no tienen valores ausentes en ninguna columna. Podemos utilizarlo para realizar cálculos o análisis adicionales, o para restaurar los valores ausentes en nuestro conjunto de datos original.
 </div>

In [43]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien

data_without_missing = data_report.dropna()
data_without_missing.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category
0,1,8437.673028,42,Bachelor Degree,0,Married,0,F,Employee,0,40620.102,purchase of the house,Adult
1,1,4024.803754,36,Secondary Education,1,Married,0,F,Employee,0,17932.802,car purchase,Adult
2,0,5623.422610,33,Secondary Education,1,Married,0,M,Employee,0,23341.752,purchase of the house,Adult
3,3,4124.747207,32,Secondary Education,1,Married,0,M,Employee,0,42820.568,supplementary education,Adult
5,0,926.185831,27,Bachelor Degree,0,Civil Partnership,1,M,Business,0,40922.170,purchase of the house,Young


In [44]:
# Examina los valores medios y medianos de los ingresos en función de los factores que identificaste

total_income_education = data_without_missing.pivot_table(index='education', values='total_income', aggfunc=['mean', 'median'])
total_income_education

,mean,median
,total_income,total_income
education,,
Bachelor Degree,33866.375230,28744.4340
Graduate Degree,27772.929500,25161.5835
Primary Education,22990.762982,20823.9590
Secondary Education,25498.606991,22673.4220
Some College,29418.625710,26113.4020


In [45]:
# Examina los valores medios y medianos de los ingresos en función de los factores que identificaste

total_income_for_type = data_without_missing.pivot_table(index='income_type', values='total_income', aggfunc=['mean', 'median'])
total_income_for_type 

,mean,median
,total_income,total_income
income_type,,
Business,32386.741818,27571.0825
Civil Servant,27343.729582,24071.6695
Employee,25820.841683,22815.1035
Entrepreneur,79866.103000,79866.1030
Paternity / Maternity Leave,8612.661000,8612.6610
Student,15712.260000,15712.2600


In [46]:
# Examina los valores medios y medianos de los ingresos en función de los factores que identificaste

total_income_gender = data_without_missing.pivot_table(index='gender', values='total_income', aggfunc=['mean', 'median'])
total_income_gender

,mean,median
,total_income,total_income
gender,,
F,25560.371100,22210.6180
M,31636.394768,27526.4765


<div style="text-align: justify">
Al examinar la disparidad entre la media y la mediana de total_income en relación a diversos factores, como educación, tipo de ingreso y género, podemos discernir qué medida es más idónea para reemplazar los valores ausentes en total_income. Estas tres categorías se consideraron específicamente debido a su influencia directa en los ingresos individuales, mientras que el estado civil se excluyó ya que no se está evaluando la influencia en los ingresos familiares.
</div>

<div style="text-align: justify">
En la tabla de "education", observamos que la diferencia entre la media y la mediana es relativamente pequeña en todas las categorías. En contraste, en la tabla de "income_type", vemos que en algunas categorías, como "Business" y "Entrepreneur", hay una diferencia significativa entre la media y la mediana. En resumen, ambas tablas ofrecen información valiosa sobre la relación entre los factores y los ingresos en "total_income". Sin embargo, considerando la diferencia entre la media y la mediana, la tabla de "income_type" podría ser más adecuada para rellenar los valores ausentes ya que proporciona una visión más clara de las posibles disparidades salariales y la presencia de valores atípicos dentro de cada categoría de ingreso. En este caso, la mediana podría ser una medida más apropiada para reemplazar los valores ausentes en "total_income".
</div>

In [47]:
#  Escribe una función que usaremos para completar los valores ausentes
        
income_type_median = data_without_missing.groupby("income_type")["total_income"].median().to_dict()

def fill_total_income(row):
    if pd.isna(row['total_income']):
        income_type = row['income_type']
        income = income_type_median.get(income_type)
        if income:
            return income
    return row['total_income']

In [48]:
# Comprueba si funciona

row_values = [np.nan, 'Business']
row_columns = ['total_income', 'income_type']

row = pd.Series(data=row_values, index=row_columns)
fill_total_income(row)

27571.0825

In [49]:
# Aplícalo a cada fila

data_report['total_income'] = data_report.apply(fill_total_income, axis=1)
data_report['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21448    35966.698
21449    24959.969
21450    14347.610
21451    39054.888
21452    13127.587
Name: total_income, Length: 21453, dtype: float64

In [50]:
# Comprueba si tenemos algún error

data_report['total_income'].isna().sum()

386

<div style="text-align: justify">
Después de aplicar la función fill_total_income, aún se encontraron 386 valores ausentes en la columna 'total_income'. Esto indica que no fue posible rellenar esos valores utilizando la mediana según la categoría de ingreso. Al revisar los datos, se observó que en estas filas los valores eran 0.00 en lugar de NaN. Por lo tanto, se decidió convertir manualmente estos valores a NaN para poder proceder con el relleno adecuado. 
</div>

> Vamos a reemplazar los valores ausentes utilizando la mediana de todos los datos disponibles en la columna 'total_income', ahora se cuenta con suficientes datos para obtener una estimación más precisa del nivel de ingresos.

In [51]:
# Reemplazar los valores ausentes si hay algún error

data_report.loc[data_report['total_income'] <= 0, 'total_income'] = np.nan
median_total_income = data_report['total_income'].median()
data_report['total_income'] = data_report['total_income'].fillna(median_total_income)
data_report['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21448    35966.698
21449    24959.969
21450    14347.610
21451    39054.888
21452    13127.587
Name: total_income, Length: 21453, dtype: float64

Vamos a comparar el número de valores en la columna 'total_income' con otras columnas para comprobar si coinciden con el número de valores, podemos utilizar el método count().

In [52]:
# Comprobar el número de entradas en las columnas

print("Filas total_income =", data_report["total_income"].count(), "Filas income_type =", data_report["income_type"].count(), "Filas education =", data_report["education"].count())

Filas total_income = 21453 Filas income_type = 21453 Filas education = 21453


###  Restaurar valores en `days_employed`

<div style="text-align: justify">
En este caso he descartado otras categorías, como la educación o el género, debido a que no están directamente relacionadas con la duración del empleo. Si bien estos factores pueden influir en otros aspectos relacionados con el empleo, no son indicativos directos de los días empleados. Por lo tanto, se procederá al estudio del factor más relevante: "income_type", vamos a investigar este aspecto para comprender mejor su relación y determinar la manera de reemplazar los valores ausentes.
</div>

In [53]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

mean_distributions = data_without_missing.groupby("income_type")["days_employed"].mean()
mean_distributions

income_type
Business                       2111.470404
Civil Servant                  3399.896902
Employee                       2326.499216
Entrepreneur                    520.848083
Paternity / Maternity Leave    3296.759962
Student                         578.751554
Name: days_employed, dtype: float64

In [54]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

median_distributions = data_without_missing.groupby("income_type")["days_employed"].median()
median_distributions

income_type
Business                       1546.333214
Civil Servant                  2689.368353
Employee                       1574.202821
Entrepreneur                    520.848083
Paternity / Maternity Leave    3296.759962
Student                         578.751554
Name: days_employed, dtype: float64

<div style="text-align: justify">
Según los datos proporcionados, la media y la mediana de la columna "days_employed" difieren en varias categorías de ingresos. En la categoría "Business", la media es de 2111.47 días mientras que la mediana es de 1546.33 días. Esto indica que la distribución de los datos puede estar sesgada hacia valores más altos, ya que la media es mayor que la mediana, lo mismo sucede en las categorías : "Civil Servant" y "Employee". En las categorías restantes, tanto la media como la mediana son similares, lo que sugiere una distribución más simétrica y menos afectada por valores atípicos.
</div>

> Por lo tanto dado que en algunas categorías hay una diferencia notable entre la media y la mediana, se procederá a utilizar la mediana para rellenar los valores ausentes en la columna "days_employed".

In [55]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado

days_employed_median = median_distributions.to_dict()

def fill_days_employed(row):
    if pd.isna(row['days_employed']):
        income_type = row['income_type']
        income = days_employed_median.get(income_type)
        if income:
            return income
    return row['days_employed']

In [56]:
# Comprueba que la función funciona

row_values = [np.nan, 'Employee']
row_columns = ['days_employed', 'income_type']

row = pd.Series(data=row_values, index=row_columns)
fill_days_employed (row)

1574.2028211070854

In [57]:
# Aplicar la función al income_type

data_report['days_employed'] = data_report.apply(fill_days_employed, axis=1)
data_report['days_employed']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4                NaN
            ...     
21448    4529.316663
21449            NaN
21450    2113.346888
21451    3112.481705
21452    1984.507589
Name: days_employed, Length: 21453, dtype: float64

In [58]:
# Comprueba si la función funcionó

data_report['days_employed'].isna().sum()

3831

In [59]:
# Reemplazar valores ausentes

data_report['days_employed'] = data_report.groupby("age_category")['days_employed'].transform(lambda x: x.fillna(x.median()))
data_report['days_employed']

0        8437.673028
1        4024.803754
2        5623.422610
3        4124.747207
4        1645.263765
            ...     
21448    4529.316663
21449    2300.843888
21450    2113.346888
21451    3112.481705
21452    1984.507589
Name: days_employed, Length: 21453, dtype: float64

<div style="text-align: justify">
Inicialmente, al considerar cómo reemplazar los valores ausentes en la columna "days_employed", identificamos que el factor más adecuado para hacerlo era "income_type". Sin embargo, posteriormente descubrimos que las categorías "retiree" y "unemployed" no disponían de datos que pudiéramos utilizar para asignar a las filas correspondientes en "days_employed". Con el objetivo de mantener la coherencia y la integridad de nuestro conjunto de datos, decidimos recurrir a una alternativa consistente. Por tanto, optamos por utilizar la mediana por "age_category" como una opción válida para rellenar los valores ausentes. 
</div>

In [60]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes

print("Filas days_employed =", data_report["days_employed"].count(), "Filas income_type =", data_report["income_type"].count(), "Filas education =", data_report["education"].count())

Filas days_employed = 21453 Filas income_type = 21453 Filas education = 21453


## Clasificación de datos

<div style="text-align: justify">
Para sustentar las hipótesis planteadas al inicio del Proyecto, se han seleccionado categorías que se considera que tienen una influencia directa en la comprobación de dichas hipótesis. Las categorías seleccionadas para su análisis son: "children" (tener hijos), "family_status" (situación familiar), "total_income" (ingreso total), "purpose" (propósito del crédito) y "debt" (deuda), esta última será utilizada para calcular la tasa de cumplimiento de pagos. El análisis de estas categorías nos permitirá evaluar la correlación entre estos factores y los pagos oportunos.
</div>

In [61]:
# Muestra los valores de los datos seleccionados para la clasificación

selected_data = data_report.loc[:, ['children', 'family_status', 'total_income', 'purpose', 'debt']]
selected_data

,children,family_status,total_income,purpose,debt
0,1,Married,40620.102,purchase of the house,0
1,1,Married,17932.802,car purchase,0
2,0,Married,23341.752,purchase of the house,0
3,3,Married,42820.568,supplementary education,0
4,0,Civil Partnership,25378.572,to have a wedding,0
...,...,...,...,...,...
21448,1,Civil Partnership,35966.698,housing transactions,0
21449,0,Married,24959.969,purchase of a car,0
21450,1,Civil Partnership,14347.610,property,1
21451,3,Married,39054.888,buying my own car,1


Vamos a revisar los valores únicos de cada categoría seleccionada para decidir de que manera clasificar sus datos.

In [62]:
# Comprobar los valores únicos

print ("Valores únicos de children:" , data_report["children"].unique())
print ()
print ("Valores únicos de family_status:" , data_report["family_status"].unique())
print ()
print ("Valores únicos de total_income:" , data_report["total_income"].unique())
print ()
print ("Valores únicos de purpose:" , data_report["purpose"].unique())
print ()
print ("Valores únicos de debt:" , data_report["debt"].unique())   

Valores únicos de children: [1 0 3 2 4 5]

Valores únicos de family_status: ['Married' 'Civil Partnership' 'Widow / Widower' 'Divorced' 'Unmarried']

Valores únicos de total_income: [40620.102 17932.802 23341.752 ... 14347.61  39054.888 13127.587]

Valores únicos de purpose: ['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 

Los grupos principales identificados en función de los valores únicos son:

- "children": Categorías que indican la cantidad de hijos.
- "family_status": Diferentes estados civiles.
- "total_income": Diferentes niveles de ingresos.
- "purpose": Diferentes propósitos de los créditos.
- "debt": Presencia o ausencia de deuda.

Vamos a proceder a clasificar en subcategorias estos grupos a fin de facilitar nuestro análisis.

In [63]:
# Escribamos una función para clasificar los datos en función de temas comunes en la columna purpose

def classify_purpose(purpose):
    housing_keywords = ['house', 'property', 'housing', 'own property']
    vehicles_keywords = ['car']
    education_keywords = ['education', 'educated', 'university']
    wedding_keywords = ['wedding']
    business_keywords = ['real estate', 'renting out']
    
    if any(keyword in purpose for keyword in housing_keywords):
        return 'Housing'
    elif any(keyword in purpose for keyword in vehicles_keywords):
        return 'Vehicles'
    elif any(keyword in purpose for keyword in education_keywords):
        return 'Education'
    elif any(keyword in purpose for keyword in wedding_keywords):
        return 'Wedding'
    elif any(keyword in purpose for keyword in business_keywords):
        return 'Business'
    else:
        return 'Other'

In [64]:
# Crea una columna con las categorías y cuenta los valores en ellas

data_report['purpose_category'] = data_report['purpose'].apply(classify_purpose)
data_report['purpose_category'].value_counts()

Housing      6347
Business     4463
Vehicles     4306
Education    4013
Wedding      2324
Name: purpose_category, dtype: int64

Ahora vamos a revisar los datos de las columnas numéricas seleccionadas "total_income" para clasificarlas en subcategorias de ingresos. 

In [65]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación

data_report["total_income"].head(10)

0    40620.102
1    17932.802
2    23341.752
3    42820.568
4    25378.572
5    40922.170
6    38484.156
7    21731.829
8    15337.093
9    23108.150
Name: total_income, dtype: float64

In [66]:
# Obtener estadísticas resumidas para la columna

data_report["total_income"].describe()

count     21453.00000
mean      26521.86431
std       15685.14033
min        3306.76200
25%       17219.35200
50%       22912.98800
75%       31331.00900
max      362496.64500
Name: total_income, dtype: float64

<div style="text-align: justify">
Para distribuir los valores de ingresos en tres categorías (Ingresos bajos, Ingresos medios e Ingresos altos), se utilizaron los percentiles como referencia. Basándonos en los valores proporcionados, se realizó la siguiente clasificación:
</div>

    - Ingresos bajos: Valores hasta el percentil 25% (17219.35200)
    - Ingresos medios: Valores entre el percentil 25% (17219.35200) y el percentil 75% (31331.00900)
    - Ingresos altos: Valores a partir del percentil 75% (31331.00900) hasta el valor máximo (362496.64500)

<div style="text-align: justify">
Esta distribución se basa en dividir los datos en tercios aproximadamente iguales, asignando el nivel bajo al tercio inferior, el nivel medio al tercio intermedio y el nivel alto al tercio superior de los valores. Esta clasificación nos permitirá analizar las diferencias en los pagos a tiempo en función de los niveles de ingresos de los individuos.
</div>

In [67]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos

def classify_income(income):
    if income <= 17219.35200:
        return "Ingresos bajos"
    elif income <= 31331.00900:
        return "Ingresos medios"
    else:
        return "Ingresos altos"

In [68]:
# Crear una columna con categorías

data_report['income_category'] = data_report['total_income'].apply(classify_income)
data_report['income_category']

0         Ingresos altos
1        Ingresos medios
2        Ingresos medios
3         Ingresos altos
4        Ingresos medios
              ...       
21448     Ingresos altos
21449    Ingresos medios
21450     Ingresos bajos
21451     Ingresos altos
21452     Ingresos bajos
Name: income_category, Length: 21453, dtype: object

In [69]:
# Contar los valores de cada categoría para ver la distribución

data_report['income_category'].value_counts()

Ingresos medios    10726
Ingresos bajos      5364
Ingresos altos      5363
Name: income_category, dtype: int64

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [70]:
# Comprueba los datos sobre los hijos y los pagos puntuales

total_clients = len (data_report["debt"])
total_debtors = data_report ["debt"].loc[data_report["debt"]==1].size
print ("Total clientes:", total_clients)
print ("Total clientes deudores:", total_debtors)
print ("Deudores con hijos:", len (data_report.loc[(data_report["debt"] == 1) & (data_report["children"] > 0)]))
print ("Deudores sin hijos:", len (data_report.loc[(data_report["debt"] == 1) & (data_report["children"] == 0)]))
print ()

# Calcular la tasa de incumplimiento en función del número de hijos

debt_by_children = data_report[data_report['debt'] == 1].groupby('children').size()
default_rate_children = debt_by_children / total_debtors * 100
print("Tasa de incumplimiento según: ", default_rate_children.apply(lambda x: "{:.2f}%".format(x)))

Total clientes: 21453
Total clientes deudores: 1741
Deudores con hijos: 677
Deudores sin hijos: 1064

Tasa de incumplimiento según:  children
0    61.11%
1    25.50%
2    11.60%
3     1.55%
4     0.23%
dtype: object


**Conclusión**

Analizando la información obtenida del conjunto de datos, podemos explorar la posible correlación entre tener hijos y pagar a tiempo. Con un total de 21,453 registros de clientes, se identificaron 1,741 clientes con deudas. De estos deudores, el 39.04% tenía hijos, mientras que el 60.96% no tenía hijos.


>Al examinar la tasa de incumplimiento según el número de hijos, se observa una tendencia interesante. Aquellos clientes sin hijos presentaron una tasa de incumplimiento del 61.11%, mientras que aquellos con un hijo tuvieron una tasa de incumplimiento del 25.50%. La tasa de incumplimiento disminuyó aún más para aquellos con más de dos hijos, llegando a la tasa más baja, con un valor del 0.23%.


Estos resultados indican que existe una posible correlación inversa entre tener hijos y pagar a tiempo. A medida que aumenta el número de hijos, la tasa de incumplimiento tiende a disminuir. 

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [71]:
# Comprueba los datos del estado familiar y los pagos a tiempo

debt_by_family_status = data_report[data_report['debt'] == 1].groupby('family_status').size()
print ("Total clientes:", total_clients)
print ("Total clientes deudores:", total_debtors)
print ()

# Calcular la tasa de incumplimiento basada en el estado familiar

default_rate_family_status = debt_by_family_status / total_debtors * 100
print("Tasa de incumplimiento según: " , default_rate_family_status.apply(lambda x: "{:.2f}%".format(x)))

Total clientes: 21453
Total clientes deudores: 1741

Tasa de incumplimiento según:  family_status
Civil Partnership    22.29%
Divorced              4.88%
Married              53.48%
Unmarried            15.74%
Widow / Widower       3.62%
dtype: object


**Conclusión**

Basado en los resultados del análisis de datos, se examinó la relación entre la situación familiar y el pago a tiempo. Del total de 21,453 clientes, se identificó que 1,741 clientes tenían deudas pendientes. Al observar la tasa de incumplimiento según la situación familiar, se encontró lo siguiente:

Entre los clientes dentro de un compromiso suman alrededor del 75% del total de la categoria deudas pendientes: 

- Los clientes casados tuvieron la tasa de incumplimiento más alta con un **53.48%**.
- Seguido por aquellos en una unión civil, con un **22.29%** en la misma categoría.

Clientes considerados sin compromiso conforman el 25% restante del total de la categoria deudas pendientes: 

- Aquí son los clientes solteros quienes presentaron la tasa más alta dentro de este grupo con un **15.74%**.
- Con un decrecimiento sustancial, encontramos a los clientes divorciados con una tasa del **4.88%**.
- Los clientes viudos tuvieron una tasa de incumplimiento de únicamente el **3.62%**.

Estos resultados sugieren que existe una correlación entre la situación familiar y el pago a tiempo, donde los clientes casados y aquellos en una unión civil parecen ser más propensos a tener deudas pendientes en comparación con los clientes divorciados, solteros y viudos. En todo caso para una análisis mas exacto podriamos revisar cuanto es la proporcion de clientes en estas categorías en el conjunto total de registros.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [72]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo

debt_by_income_category = data_report[data_report['debt'] == 1].groupby('income_category').size()
print ("Total clientes:", total_clients)
print ("Total clientes deudores:", total_debtors)
print ()

# Calcular la tasa de incumplimiento basada en el nivel de ingresos

default_rate_income = debt_by_income_category / total_debtors * 100
print("Tasa de incumplimiento según: " , default_rate_income.apply(lambda x: "{:.2f}%".format(x)))

Total clientes: 21453
Total clientes deudores: 1741

Tasa de incumplimiento según:  income_category
Ingresos altos     22.00%
Ingresos bajos     24.53%
Ingresos medios    53.48%
dtype: object


**Conclusión**
<div style="text-align: justify">
Durante el análisis de crédito, se evaluó la posible correlación entre el nivel de ingresos y el pago a tiempo de los clientes. Con base en los resultados obtenidos, se observaron diferencias significativas en la tasa de incumplimiento según la categoría de ingresos.
</div>

- En primer lugar, se identificó que los clientes con ingresos medios presentaron la tasa de incumplimiento más alta, con un preocupante 53.48%. Esto indica que existe una propensión significativa de este grupo a no cumplir con sus pagos de manera puntual. Es importante resaltar que estos los grupos de ingresos medios y bajos representan una preocupación destacada en términos de riesgo crediticio.

<div style="text-align: justify"> 
Estos hallazgos indican la existencia de una relación entre el nivel de ingresos y la puntualidad en los pagos. Sin embargo, es fundamental considerar otros factores que pueden influir en esta relación y afectar la capacidad de pago de los clientes
</div>

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [73]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito

print("Total de clientes:", total_clients)
print("Total de clientes deudores:", total_debtors)
print()
debt_by_purpose_credit = data_report[data_report['debt'] == 1].groupby('purpose_category').size()

# Calcular la tasa de incumplimiento basada en el propósito del crédito

default_rate_purpose = debt_by_purpose_credit / total_debtors * 100
print("Tasa de incumplimiento según: " , default_rate_purpose.apply(lambda x: "{:.2f}%".format(x)))

Total de clientes: 21453
Total de clientes deudores: 1741

Tasa de incumplimiento según:  purpose_category
Business     19.30%
Education    21.25%
Housing      25.62%
Vehicles     23.15%
Wedding      10.68%
dtype: object


**Conclusión**

El propósito del crédito tiene un impacto significativo en la tasa de incumplimiento de los pagos. Al analizar diferentes categorías de propósito, encontramos que los préstamos destinados a "Housing" (vivienda) presentan la tasa de incumplimiento más alta, con un 25.62%. Esto puede deberse a los compromisos financieros considerables y a largo plazo asociados con la adquisición o mantenimiento de una vivienda.

Por otro lado, los préstamos para "Wedding" (bodas) muestran la tasa de incumplimiento más baja, con un 10.68%. Este resultado podría explicarse por la importancia personal y emocional de las bodas, lo que podría motivar a los prestatarios a hacer un mayor esfuerzo para cumplir con sus pagos y evitar el incumplimiento.

# Conclusión general 

Este análisis de crédito nos permitió obtener información valiosa sobre el comportamiento de los clientes en relación con el pago de sus préstamos. Durante el proceso de análisis, enfrentamos desafíos comunes en la limpieza y preparación de los datos, incluyendo valores ausentes, duplicados y artefactos problemáticos.

Uno de los desafíos más comunes fue lidiar con los valores ausentes en diferentes variables. Implementamos estrategias para reemplazar los valores faltantes de manera adecuada, utilizando medianas condicionales basadas en diferentes categorías, como el tipo de ingreso y la categoría de edad. Esto nos permitió mantener la integridad de los datos y obtener resultados más precisos en nuestros análisis.

Otro desafío fue la detección y manejo de duplicados en el conjunto de datos. Utilizamos métodos como la función duplicated() para identificar y eliminar duplicados, asegurando así que nuestros análisis se realizaran en datos únicos y evitando distorsiones en los resultados.

Además, encontramos artefactos problemáticos en los datos, como valores atípicos o incoherentes. Estos artefactos pueden afectar negativamente nuestros análisis y conclusiones. Implementamos técnicas de detección de valores atípicos y evaluamos su impacto en nuestros resultados. En algunos casos, realizamos ajustes o eliminamos los datos problemáticos para obtener una imagen más precisa y confiable.

En resumen, al abordar los desafíos de valores ausentes, duplicados y artefactos problemáticos, pudimos procesar y analizar los datos de manera efectiva. Esto nos permitió obtener conclusiones relevantes sobre la correlación entre variables como tener hijos, situación familiar, nivel de ingresos y propósito del crédito con el pago puntual. Estas conclusiones son fundamentales para comprender los factores que influyen en el cumplimiento de los préstamos y pueden ser utilizadas para tomar decisiones informadas en el análisis de riesgo crediticio.

>En base a las preguntas planteadas y el análisis realizado, podemos extraer las siguientes conclusiones:

- Existe una correlación entre tener hijos y el pago a tiempo. Según los datos, se observa que la tasa de incumplimiento disminuye a medida que aumenta el número de hijos. Los clientes sin hijos presentan una tasa de incumplimiento del 4.96%, mientras que aquellos con un hijo tienen una tasa del 2.07%, y aquellos con dos o más hijos presentan tasas aún más bajas. Esto sugiere que tener hijos puede influir positivamente en el pago puntual de los préstamos.

- Existe una correlación entre la situación familiar y el pago a tiempo. Los datos muestran que el estado civil de los clientes está relacionado con la tasa de incumplimiento. Los clientes casados tienen la tasa más alta de incumplimiento, con un 4.34%, seguidos por los clientes solteros, con una tasa del 1.28%. Los clientes en una asociación civil, divorciados y viudos tienen tasas de incumplimiento aún más bajas. Esto indica que la situación familiar puede ser un factor importante a considerar en la evaluación del riesgo crediticio.

- Existe una correlación entre el nivel de ingresos y el pago a tiempo. El análisis revela que los clientes con ingresos más altos presentan una tasa de incumplimiento más baja en comparación con aquellos con ingresos bajos o medios. Los clientes con ingresos altos tienen una tasa de incumplimiento del 22.00%, mientras que aquellos con ingresos bajos y medios tienen tasas del 24.53% y 53.48%, respectivamente. Esto sugiere que el nivel de ingresos puede ser un indicador de la capacidad de pago y la responsabilidad financiera de los clientes.

El propósito del crédito también influye en la tasa de incumplimiento. Según el análisis, se observa que los préstamos para vivienda tienen la tasa de incumplimiento más alta, con un 25.62%. Por otro lado, los préstamos para bodas tienen la tasa más baja, con un 10.68%. Los préstamos para negocios, educación y vehículos se encuentran en un rango intermedio. Esto destaca la importancia de evaluar el propósito del crédito al analizar el riesgo de incumplimiento.
